In [1]:
# импортируем библиотеки numpy и pandas
import pandas as pd
import numpy as np

# импортируем библиотеку datetime для работы с датами
from datetime import datetime, date

# Задаем некоторые опции библиотеки pandas, которые настраивают вывод
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 10) 
pd.set_option('display.max_rows', 15)
pd.set_option('display.width', 90)

# импортируем библиотеку matplotlib для построения графиков
import matplotlib.pyplot as plt
%matplotlib inline

#### Конкатенация данных

In [2]:
# создаем два объекта Series для конкатенации
s1 = pd.Series(np.arange(0, 3))
s2 = pd.Series(np.arange(5, 8))
s1, s2

(0    0
 1    1
 2    2
 dtype: int64,
 0    5
 1    6
 2    7
 dtype: int64)

In [3]:
# конкатенируем их
pd.concat([s1, s2])

0    0
1    1
2    2
0    5
1    6
2    7
dtype: int64

Вышеприведенный программный код присоединяет метки индекса и значения объекта s2 к конечной метке индекса и конечному значению объекта s1. В результате **получаем дублирующиеся индексные метки**, поскольку в ходе этой операции выравнивание не выполняется.

In [4]:
# создаем два объекта DataFrame для конкатенации, используя те же самые индексные метки
# и имена столбцов, но другие значения
df1 = pd.DataFrame(np.arange(9).reshape(3, 3), columns=['a', 'b', 'c'])
df2 = pd.DataFrame(np.arange(9, 18).reshape(3, 3), columns=['a', 'b', 'c'])
df1, df2

(   a  b  c
 0  0  1  2
 1  3  4  5
 2  6  7  8,
     a   b   c
 0   9  10  11
 1  12  13  14
 2  15  16  17)

In [5]:
# выполняем конкатенацию
# в результате мы получим дублирующиеся индексные метки вдоль индекса строк
pd.concat([df1, df2])

    a   b   c
0   0   1   2
1   3   4   5
2   6   7   8
0   9  10  11
1  12  13  14
2  15  16  17

Следующий программный код демонстрирует выравнивание двух объектов
DataFrame в ходе конкатенации, когда есть общие столбцы (a и c), а также столбцы,
присутствующие только в одном датафрейме (столбец b в датафрейме df1 и столбец d в датафрейме df2).

In [6]:
# демонстрируем конкатенацию двух объектов DataFrame с разными столбцами
df1 = pd.DataFrame(np.arange(9).reshape(3, 3), columns=['a', 'b', 'c'])
df2 = pd.DataFrame(np.arange(9, 18).reshape(3, 3), columns=['a', 'c', 'd'])
df1, df2

(   a  b  c
 0  0  1  2
 1  3  4  5
 2  6  7  8,
     a   c   d
 0   9  10  11
 1  12  13  14
 2  15  16  17)

In [7]:
# выполняем конкатенацию, пропусками будут заполнены значения столбца d в датафрейме df1
# и значения столбца b в датафрейме df2
pd.concat([df1, df2])

    a    b   c     d
0   0  1.0   2   NaN
1   3  4.0   5   NaN
2   6  7.0   8   NaN
0   9  NaN  10  11.0
1  12  NaN  13  14.0
2  15  NaN  16  17.0

Каждой группе данных в итоговом датафрейме можно дать свое название, воспользовавшись параметром **keys**. *Он создает иерархический индекс объекта DataFrame, который позволяет отдельно обращаться к каждой группе данных с помощью свойства .loc объекта DataFrame*. Это удобно, если позже в итоговом объекте DataFrame вам потребуется определить источник получения данных.

In [8]:
# выполняем конкатенацию двух объектов, но при этом создаем индекс с помощью
# заданных ключей
c = pd.concat([df1, df2], keys=['df1', 'df2'])
# обратите внимание на метки строк в выводе
c

        a    b   c     d
df1 0   0  1.0   2   NaN
    1   3  4.0   5   NaN
    2   6  7.0   8   NaN
df2 0   9  NaN  10  11.0
    1  12  NaN  13  14.0
    2  15  NaN  16  17.0

In [9]:
# мы можем извлечь данные, относящиеся к первому или второму исходному датафрейму
c.loc['df2']

    a   b   c     d
0   9 NaN  10  11.0
1  12 NaN  13  14.0
2  15 NaN  16  17.0

#### Переключение осей выравнивания
Функция pd.concat() позволяет задать ось, к которой нужно применить выравнивание во время конкатенации.


In [10]:
# конкатенируем датафреймы df1 и df2 по оси столбцов
# выравниваем по меткам строк, получаем дублирующиеся столбцы
pd.concat([df1, df2], axis='columns')

   a  b  c   a   c   d
0  0  1  2   9  10  11
1  3  4  5  12  13  14
2  6  7  8  15  16  17

В результате получаем дублирующиеся столбцы. Это связано с тем, что в ходе
конкатенации **сначала происходит выравнивание по меткам индекса строк** каждого объекта DataFrame, а затем осуществляется заполнение значений столбцов для первого объекта DataFrame и второго объекта DataFrame независимо от меток индекса строк.

In [11]:
# создаем новый датафрейм d3, чтобы конкатенировать его с датафреймом df1
# датафрейм df3 имеет общую с датафреймом df1 метку 2 и общий столбец a
df3 = pd.DataFrame(np.arange(20, 26).reshape(3, 2),
                  columns=['a', 'd'],
                  index=[2, 3, 4])
df3

    a   d
2  20  21
3  22  23
4  24  25

In [12]:
# конкатерируем их по оси столбцов. Происходит выравнивание по меткам строк, осуществляется
# заполнение значений столбцов df1, а затем столбцов df3, получаем дублирующиеся столбцы
pd.concat([df1, df3], axis='columns')

     a    b    c     a     d
0  0.0  1.0  2.0   NaN   NaN
1  3.0  4.0  5.0   NaN   NaN
2  6.0  7.0  8.0  20.0  21.0
3  NaN  NaN  NaN  22.0  23.0
4  NaN  NaN  NaN  24.0  25.0

Поскольку выравнивание осуществлялось по меткам индекса строк, мы получаем дублирующиеся столбцы. Значения для меток строк 0 и 1 в столбцах a и d, соответствующих датафрейму df3, заполняются пропусками, поскольку эти метки есть только в датафрейме df1. Значения для меток строк 3 и 4 в столбцах a, b и c, соответствующих датафрейму df1, заполняются пропусками, поскольку эти метки
есть только в датафрейме df3.

#### Определение типа соединения

In [13]:
# выполняем внутреннее соединение вместо внешнего
# результат представлен в виде одной строки
pd.concat([df1, df3], axis='columns', join='inner')

   a  b  c   a   d
2  6  7  8  20  21

In [14]:
# добавляем ключи к столбцам
df = pd.concat([df1, df3], keys=['df1', 'df2'], axis='columns', join='inner')
df

  df1       df2    
    a  b  c   a   d
2   6  7  8  20  21

In [15]:
# извлекаем данные из датафрейма с помощью ключа 'df2'
df.loc[:, 'df2']

    a   d
2  20  21

#### Игнорирование меток индекса
Если вы хотите избавиться от дублирования меток в итоговом индексе и при этом
сохранить все строки, вы можете воспользоваться параметром **ignore_index=True**.
Это, по сути, возвращает тот же результат, за исключением того, что теперь наш
индекс имеет тип Int64Index

In [16]:
# избавляемся от дублирования меток в итоговом индексе,
# игнорируя индексные метки в датафреймах-источниках
pd.concat([df1, df2], ignore_index=True)

    a    b   c     d
0   0  1.0   2   NaN
1   3  4.0   5   NaN
2   6  7.0   8   NaN
3   9  NaN  10  11.0
4  12  NaN  13  14.0
5  15  NaN  16  17.0

#### Слияние и соединение данных
Библиотека pandas позволяет выполнить слияние объектов с помощью операций, аналогичных операциям соединения для баз данных, используя функцию **pd.merge()** и метод **.merge()** объекта DataFrame. 

Процедура слияния объединяет данные двух объектов путем поиска совпадающих значений в одном или нескольких индексах столбцов или строк. Затем, применив семантику соединения к этим
значениям, она возвращает новый объект – комбинацию данных обоих объектов.

Слияние полезно, поскольку оно позволяет нам создавать отдельный объект DataFrame для каждого типа данных (одно из правил, позволяющее получить аккуратные данные), а также связывать данные в разных объектах DataFrame с помощью значений, присутствующих в обоих наборах.

#### Слияние данных, расположенных в нескольких объектах
Практический пример использования слияния – это поиск имен и адресов клиентов в заказах.

In [17]:
# это наши клиенты
customers = {'CustomerID': [10, 11],
            'Name': ['Mike', 'Marica'],
            'Address': ['Address for Mike', 'Address for Marcia']}
orders = {'CustomerID': [10, 11, 10],
            'OrderDate': [date(2014, 12, 1), date(2014, 12, 1), date(2014, 12, 1)]
         }

customers = pd.DataFrame(customers)
orders = pd.DataFrame(orders)

customers, '-------', orders

(   CustomerID    Name             Address
 0          10    Mike    Address for Mike
 1          11  Marica  Address for Marcia,
 '-------',
    CustomerID   OrderDate
 0          10  2014-12-01
 1          11  2014-12-01
 2          10  2014-12-01)

In [18]:
# выполняем слияние датафреймов customers и orders так, чтобы мы могли отправить товары
customers.merge(orders)

   CustomerID    Name             Address   OrderDate
0          10    Mike    Address for Mike  2014-12-01
1          10    Mike    Address for Mike  2014-12-01
2          11  Marica  Address for Marcia  2014-12-01

Библиотека pandas совершила для нас какое-то волшебство, выполнив слияние данных с помощью такого простого кода. Она поняла, что у наших датафреймов customers и orders есть общий столбец CustomerID и с этими знаниями. Затем она воспользовалась общими значениями, найденными в столбце CustomerID обоих датафреймов, чтобы связать данные друг с другом и сформировать итоговые данные на основе семантики внутреннего соединения.

In [19]:
# создаем данные, которые будем использовать в качестве примеров
# в оставшейся части этого раздела
left_data = {'key1': ['a', 'b', 'c'],
            'key2': ['x', 'y', 'z'],
            'lval1': [ 0, 1, 2]}

right_data = {'key1': ['a', 'b', 'c'],
            'key2': ['x', 'a', 'z'],
            'rval1': [ 6, 7, 8 ]}
left = pd.DataFrame(left_data)
right = pd.DataFrame(right_data)

left, '-----', right

(  key1 key2  lval1
 0    a    x      0
 1    b    y      1
 2    c    z      2,
 '-----',
   key1 key2  rval1
 0    a    x      6
 1    b    a      7
 2    c    z      8)

In [20]:
# демонстрируем слияние, не указывая столбцы, по которым нужно выполнить слияние
# этот программный код неявно выполняет слияние по всем общим столбцам
left.merge(right)

  key1 key2  lval1  rval1
0    a    x      0      6
1    c    z      2      8

Процедура слияния определяет, что столбцы **key1 и key2 являются общими для обоих объектов DataFrame**. Соответствующие кортежи значений в обоих объектах DataFrame для этих столбцов выглядят как (a, x) и (c, z), поэтому в результате получаем две строки значений.

Чтобы явно задать столбец, используемый для связывания объектов, можно воспользоваться параметром **on**. Следующий программный код выполняет слияние, **используя только значения столбца key1 обоих объектов DataFrame**

In [21]:
# демонстрируем слияние, явно задав столбец, по значениям которого нужно связать
# объекты DataFrame
left.merge(right, on='key1', suffixes=['_l', '_r'])

  key1 key2_l  lval1 key2_r  rval1
0    a      x      0      x      6
1    b      y      1      a      7
2    c      z      2      z      8

In [22]:
# явно выполняем слияние с помощью двух столбцов
left.merge(right, on=['key1', 'key2'])

  key1 key2  lval1  rval1
0    a    x      0      6
1    c    z      2      8

Если вы хотите выполнить слияние на основе столбцов с **разными именами в каждом объекте**, то можете использовать параметры **left_on** и **right_on**, передав имя или имена столбцов каждому соответствующему параметру.

**Т.е мы сами решаем какой столбец брать из левого и правого датафрейма и результатом будут строки, имеющие одинаковые значения.**

In [23]:
left.merge(right, left_on='key1', right_on='key2')

  key1_x key2_x  lval1 key1_y key2_y  rval1
0      a      x      0      b      a      7

Чтобы выполнить слияние с помощью индексных меток строк обоих объектов DataFrame, можно воспользоваться параметрами **left_index=True и right_index=True (нужно задать оба параметра)**

In [24]:
# соединяем индексы строк обеих матриц
pd.merge(left, right, left_index=True, right_index=True, suffixes=['_l', '_r'])

  key1_l key2_l  lval1 key1_r key2_r  rval1
0      a      x      0      a      x      6
1      b      y      1      b      a      7
2      c      z      2      c      z      8

#### Настройка семантики соединения при выполнении слияния
Тип соединения, выполняемый функцией pd.merge() по умолчанию, – внутреннее соединение (**inner join**). Чтобы использовать другой тип соединения, укажите его с помощью параметра **how** функции pd.merge() (или метода .merge()). Допустимыми параметрами являются:

- inner: выполняет пересечение ключей из обоих объектов DataFrame;
- outer: выполняет объединение ключей из обоих объектов DataFrame;
- left: использует только ключи из левого объекта DataFrame;
- right: использует только ключи из правого объекта DataFrame.

In [25]:
# внешнее соединение возращает все строки из внутреннего соединения,
# а также строки датафреймов left и right, не попавшие во внутреннее соединение
# отсутствующие элементы заполняются значениями NaN

In [26]:
left.merge(right, how='outer')

  key1 key2  lval1  rval1
0    a    x    0.0    6.0
1    b    a    NaN    7.0
2    b    y    1.0    NaN
3    c    z    2.0    8.0

Левое соединение возвращает все строки, попавшие во внутреннее соединение (соединенные строки датафреймов left и right, у которых совпали значения в **общих столбцах key1 и key2**), плюс все строки из датафрейма left, не попавшие во внутреннее соединение (для которых не нашлось пары в датафрейме right).

In [27]:
# левое соединение возвращает все строки из внутреннего соединения,
# а также строки датафрейма left, не попавшие во внутреннее соединение
# отсутствующие элементы заполняются значениями NaN
# итоговый датафрейм содержит общие строки датафреймов left и right с метками 0 и 2 (строки
# датафреймов left и right, у которых совпали значения в общих столбцах key1 и key2)
# а также уникальную строку датафрейма left с меткой 1
# уникальная строка датафрейма left в итоговом датафрейме в столбце rval1 заполняется
# значением NaN, потому что в датафрейме left этот столбец отсутствовал
left.merge(right, how='left')

  key1 key2  lval1  rval1
0    a    x      0    6.0
1    b    y      1    NaN
2    c    z      2    8.0

In [28]:
# правое соединение возращает все строки из внутреннего соединения,
# а также строки датафрейма right, не попавшие во внутреннее соединение
# отсутствующие элементы заполняются значениями NaN
# итоговый датафрейм содержит общие строки датафреймов left и right с метками 0 и 1 (строки
# датафреймов left и right, у которых совпали значения в общих столбцах key1 и key2)
# а также уникальную строку датафрейма right с меткой 2
# уникальная строка датафрейма right в итоговом датафрейме в столбце lval1 заполняется
# значением NaN, потому что в датафрейме right этот столбец отсутствовал

right, left.merge(right, how='right')

(  key1 key2  rval1
 0    a    x      6
 1    b    a      7
 2    c    z      8,
   key1 key2  lval1  rval1
 0    a    x    0.0      6
 1    b    a    NaN      7
 2    c    z    2.0      8)

**Кроме того, библиотека pandas предлагает метод .join()**, который можно использовать для **выполнения соединения с помощью <u>индексных меток</u>** двух объектов DataFrame (вместо значений столбцов).

Обратите внимание, что если столбцы в обоих объектах DataFrame не имеют уникальных имен, вы должны указать суффиксы с помощью параметров lsuffix и rsuffix (в отличие от слияния, автоматический суффикс при выполнении этой операции не присваивается). 


In [29]:
# соединяем left с right (метод по умолчанию – outer), и поскольку эти объекты имеют
# дублирующиеся имена столбцов, мы задаем параметы lsuffix и rsuffix
left.join(right, lsuffix='_left', rsuffix='_right')

  key1_left key2_left  lval1 key1_right key2_right  rval1
0         a         x      0          a          x      6
1         b         y      1          b          a      7
2         c         z      2          c          z      8

Тип соединения, используемый по умолчанию, – внешнее соединение. Обратите внимание, что это отличается от стандартного метода **.merge(), в котором по умолчанию применяется внутреннее соединение**.

In [30]:
# соединяем left с right с помощью внутреннего соединения
left.join(right, lsuffix='_left', rsuffix='_right', how='inner')

  key1_left key2_left  lval1 key1_right key2_right  rval1
0         a         x      0          a          x      6
1         b         y      1          b          a      7
2         c         z      2          c          z      8

#### Поворот данных для преобразования значений в индексы и наоборот
Данные часто хранятся в состыкованном формате («в столбик»), который еще называют форматом записи. Он используется в базах данных, CSV-файлах и таблицах Excel. В состыкованном формате данные часто не нормированы и имеют повторяющиеся значения в нескольких столбцах или значения, которые логически должны относиться к другими таблицам (нарушая концепцию аккуратных
данных).

In [31]:
# считываем данные акселерометра
sensor_radings = pd.read_csv('Notebooks/Data/accel.csv')
sensor_radings

    interval axis  reading
0          0    X      0.0
1          0    Y      0.5
2          0    Z      1.0
3          1    X      0.1
4          1    Y      0.4
5          1    Z      0.9
6          2    X      0.2
7          2    Y      0.3
8          2    Z      0.8
9          3    X      0.3
10         3    Y      0.2
11         3    Z      0.7

**Проблема** такого представления данных заключается в том, что довольно **трудно выделить показания, относящиеся к конкретной оси**. Это можно сделать с помощью логического отбора.

In [32]:
# извлекаем показания по оси X
sensor_radings[sensor_radings['axis'] == 'X']

   interval axis  reading
0         0    X      0.0
3         1    X      0.1
6         2    X      0.2
9         3    X      0.3

Проблема возникает, когда вам нужно узнать значения по всем осям в данный
интервал времени, а не только по оси x. Чтобы получить значения по всем осям,
можно выполнить отбор каждого значения оси, но это будет громоздкий код.

Более оптимальным станет такое представление данных, в котором столбцы будут представлять уникальные значения переменной axis. Чтобы преобразовать данные в этот формат, используйте метод <mark>.pivot()</mark> объекта DataFrame

In [33]:
# поворачиваем данные. Интервалы становятся индексом, столбцы –
# это оси, а показания – значения столбцов

In [53]:
sensor_radings['new'] = np.random.randint(1, 100, size=len(sensor_radings))
sensor_radings.pivot(index='interval',
                    columns='axis',
                    values='reading')

axis        X    Y    Z
interval               
0         0.0  0.5  1.0
1         0.1  0.4  0.9
2         0.2  0.3  0.8
3         0.3  0.2  0.7

#### Состыковка и расстыковка данных
Методы <mark>.stack() и .unstack()</mark> схожи с методом .pivot(). **Процесс состыковки поворачивает уровень меток столбцов, превращая его в индекс строк**. Расстыковка выполняет обратное действие, то есть поворачивает уровень индекса строк, превращая его в индекс столбцов.

Одно из различий между состыковкой/расстыковкой и поворотом заключается в том, что, в отличие от поворота, операции состыковки и расстыковки могут повернуть конкретные уровни иерархического индекса. Кроме того, если поворот сохраняет одинаковое количество уровней индекса, состыковка и расстыковка всегда увеличивают количество уровней по индексу одной оси (количество столбцов для расстыковки и количество строк для состыковки) и уменьшают количество уровней по другой оси.

#### Состыковка с помощью неиерархических индексов

In [35]:
# создаем простой датафрейм с одним столбцом
df = pd.DataFrame({'a': [1, 2]}, index=['one', 'two'])
df

     a
one  1
two  2

Состыковка помещает **уровень индекса столбцов в <u>новый</u> уровень индекса строк**. Поскольку наш объект DataFrame имеет только один уровень, происходит сворачивание объекта DataFrame в объект Series с иерархическим индексом строк.

In [36]:
# помещаем столбец в еще один уровень индекса строк
# результатом становится объект Series, в котором значения можно просмотреть
# с помощью мультииндекса
stacked1 = df.stack()
stacked1

one  a    1
two  a    2
dtype: int64

Чтобы посмотреть значения, нам нужно передать кортеж в индексатор объекта
Series, который выполняет поиск с помощью индекса

In [37]:
# ищем значение для 'one'/'a', передав кортеж в индексатор
stacked1[('one', 'a')]

1

Если объект DataFrame содержит несколько столбцов, то **все столбцы** помещаются
**в один и тот же дополнительный уровень** нового объекта Series

In [38]:
# создаем датафрейм с двумя столбцами
df = pd.DataFrame({'a': [1, 2], 'b': [3, 4], 'c': [5, 6]}, index=['one', 'two'])
df

     a  b  c
one  1  3  5
two  2  4  6

In [39]:
# помещаем оба столбца в отдельный уровень индекса
stackd2 = df.stack()
stackd2

one  a    1
     b    3
     c    5
two  a    2
     b    4
     c    6
dtype: int64

In [40]:
# ищем значение с помощью индекса 'one' / 'b'
stackd2[('one', 'b')]

3

#### Расстыковка с помощью иерархических индексов
**Расстыковка выполняет противоположную операцию, помещая уровень индекса строк в уровень оси столбцов.**

Чтобы продемонстрировать расстыковку с помощью иерархических индексов, мы обратимся к данным акселерометра, которые мы уже рассматривали ранее в этой главе. Однако теперь мы создадим две копии этих данных, по одной для каждого пользователя, и в каждую копию добавим дополнительный столбец. Следующий программный код создает эти данные

In [41]:
# создаем две копии данных акселерометра, по одной для каждого пользователя
user1 = sensor_radings.copy()
user2 = sensor_radings.copy()

# добавляем столбец who в каждую копию
user1['who'] = 'Mike'
user2['who'] = 'Mikael'

# давайте отмасштабируем данные user2
user2['reading'] *= 100

# и организуем данные так, чтобы получить иерархический индекс строк
multy_user_sensor_data = pd.concat([user1, user2]).set_index(['who', 'interval', 'axis'])
multy_user_sensor_data

                      reading  new
who    interval axis              
Mike   0        X         0.0   33
                Y         0.5   22
                Z         1.0   83
       1        X         0.1   16
                Y         0.4   89
...                       ...  ...
Mikael 2        Y        30.0   61
                Z        80.0   55
       3        X        30.0    9
                Y        20.0   62
                Z        70.0   96

[24 rows x 2 columns]

In [42]:
# извлекаем все показания по всем осям (поле axis) и по всем пользователям по значению интервала === 1
# (т.е это многомерный loc т.е мы хотем получть строчки со зн. 1 в индексе interval )
multy_user_sensor_data.xs(1, level='interval')

             reading  new
who    axis              
Mike   X         0.1   16
       Y         0.4   89
       Z         0.9   12
Mikael X        10.0   16
       Y        40.0   89
       Z        90.0   12

**Расстыковка помещает самый внутренний уровень индекса строк в новый уровень индекса столбцов**, в результате получаем столбцы с типом индекса MultiIndex.

In [43]:
# выполняем расстыковку, в результате самый внутренний уровень индекса строк (уровень axis)
# стал уровнем индекса столбцов
multy_user_sensor_data.unstack()

                reading              new        
axis                  X     Y      Z   X   Y   Z
who    interval                                 
Mikael 0            0.0  50.0  100.0  33  22  83
       1           10.0  40.0   90.0  16  89  12
       2           20.0  30.0   80.0  84  61  55
       3           30.0  20.0   70.0   9  62  96
Mike   0            0.0   0.5    1.0  33  22  83
       1            0.1   0.4    0.9  16  89  12
       2            0.2   0.3    0.8  84  61  55
       3            0.3   0.2    0.7   9  62  96

Чтобы выполнить расстыковку другого уровня, используйте параметр level

In [44]:
# выполняем расстыковку по уровню 0
multy_user_sensor_data.unstack(level=0)

              reading         new     
who            Mikael Mike Mikael Mike
interval axis                         
0        X        0.0  0.0     33   33
         Y       50.0  0.5     22   22
         Z      100.0  1.0     83   83
1        X       10.0  0.1     16   16
         Y       40.0  0.4     89   89
         Z       90.0  0.9     12   12
2        X       20.0  0.2     84   84
         Y       30.0  0.3     61   61
         Z       80.0  0.8     55   55
3        X       30.0  0.3      9    9
         Y       20.0  0.2     62   62
         Z       70.0  0.7     96   96

In [45]:
# выполняем расстыковку уровней who и axis
unstacked = multy_user_sensor_data.unstack(['who', 'axis'])
unstacked

         reading                         ...  new                   
who         Mike           Mikael        ... Mike     Mikael        
axis           X    Y    Z      X     Y  ...    Y   Z      X   Y   Z
interval                                 ...                        
0            0.0  0.5  1.0    0.0  50.0  ...   22  83     33  22  83
1            0.1  0.4  0.9   10.0  40.0  ...   89  12     16  89  12
2            0.2  0.3  0.8   20.0  30.0  ...   61  55     84  61  55
3            0.3  0.2  0.7   30.0  20.0  ...   62  96      9  62  96

[4 rows x 12 columns]

In [46]:
# мы можем выполнить состыковку уровней, 
# которые расстыковали выполняем состыковку уровня who
unstacked.stack(level='who', future_stack=True)

                reading              new        
axis                  X     Y      Z   X   Y   Z
interval who                                    
0        Mike       0.0   0.5    1.0  33  22  83
         Mikael     0.0  50.0  100.0  33  22  83
1        Mike       0.1   0.4    0.9  16  89  12
         Mikael    10.0  40.0   90.0  16  89  12
2        Mike       0.2   0.3    0.8  84  61  55
         Mikael    20.0  30.0   80.0  84  61  55
3        Mike       0.3   0.2    0.7   9  62  96
         Mikael    30.0  20.0   70.0   9  62  96

#### Расплавление данных для преобразования «широкого» формата в «длинный» и наоборот

**Расплавление** - это тип реорганизации данных, который часто называют преобразованием
объекта DataFrame из "широкого" формата (wide format) в "длинный" формат (long format).

Этот формат является общепринятым при проведении различных видов статистического анализа, 
и данные, которые вы считываете, могут быть представлены уже в расплавленном виде.
Либо вам, возможно, потребуется преобразовать данные в этот формат.

С технической точки зрения расплавление - это процесс изменения формы
объекта DataFrame, в результате получаем формат, в котором два или более столбцов,
названных variable и value, создаются путем сворачивания меток столбцов
исходного датафрейма в столбец variable, а затем происходит перемещение значений
из этих столбцов в соответствующую позицию столбца value. Все остальные
столбцы превращаются в столбцы-идентификаторы, которые помогают описать
данные.

In [47]:
# продемонстрируем расплавление с помощью этого датафрейма
data = pd.DataFrame({'Name' : ['Mike', 'Mikael'],
                    'Height' : [6.1, 6.0],
                    'Weight' : [220, 185]})
data

     Name  Height  Weight
0    Mike     6.1     220
1  Mikael     6.0     185

Следующий программный код расплавляет этот датафрейм, используя столбец Name в качестве 
столбца-идентификатора, а столбцы Height и Weight в качестве переменных.
Столбец Name остается неизменным, при этом столбцы Height и Weight сворачиваются
в столбец variable. Затем значения этих двух столбцов становятся значениями
столбца value и соответствуют комбинациям значений переменных Name и variable.

In [48]:
# расплавляем датафрейм, используем Name в качестве идентификатора, а столбцы
# Height and Weight в качестве переменных
pd.melt(data, id_vars=['Name'], value_vars=['Height', 'Weight'])

     Name variable  value
0    Mike   Height    6.1
1  Mikael   Height    6.0
2    Mike   Weight  220.0
3  Mikael   Weight  185.0

Данные теперь реструктурированы, поэтому легко извлечь значение для любой
комбинации переменных variable и Name. Кроме того, в таком формате представления
проще добавить новую переменную и наблюдение, поскольку можно просто
добавить данные в виде новой строки, и для этого не требуется менять структуру
датафрейма, добавляя новый столбец.

#### Преимущества использования состыкованных данных

Наконец, мы выясним, зачем нужно состыковывать данные. Можно продемонстрировать,
что поиск значений в состыкованных данных более эффективен, если сравнивать его с поиском значений по отдельному уровню индекса, а затем по столбцу и даже с поиском на основе свойства .iloc, который определяет строку и столбец по месторасположению.

In [52]:
df, stacked1

(     a  b  c
 one  1  3  5
 two  2  4  6,
 one  a    1
 two  a    2
 dtype: int64)

In [51]:
# поиск значений в состыкованных данных может быть намного быстрее поиска в обычных данных
# время выполнения различных методов
import timeit
t = timeit.Timer("stacked1[('one', 'a')]", "from __main__ import stacked1, df")
r1 = timeit.timeit(lambda: stacked1.loc[('one', 'a')], number=10000)
r2 = timeit.timeit(lambda: df.loc['one']['a'], number=10000)
r3 = timeit.timeit(lambda: df.iloc[1, 0], number=10000)

# и вот наши результаты... Да, это самый быстрый метод из всех трех
r1, r2, r3

(0.41131835099986347, 0.2705063749999681, 0.23601937700004783)